## TensorFlow
One of the main reasons for success of neural networks is the introduction of new software packages designed specifically for creating and training neural network models. These packages have the capability to train the model on specific hardware such as CPUs, GPUs or even TPUs.  

In this notebook, we use TensorFlow to train a neural network model. The other main package for neural networks is Torch. Its python wrapper pyTorch.  
First, we need to install the package on the machine.

In [2]:
!pip3 install tensorflow

     |████████████████████████████████| 199.0 MB 35.6 MB/s eta 0:00:01   |██████████▎                     | 63.6 MB 7.2 MB/s eta 0:00:19     |█████████████████▊              | 110.2 MB 24.6 MB/s eta 0:00:04     |███████████████████████▊        | 147.6 MB 65.6 MB/s eta 0:00:01███████████████▊| 197.2 MB 35.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.9 MB 60.1 MB/s eta 0:00:01
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     |████████████████████████████████| 1.3 MB 50.3 MB/s eta 0:00:01
     |████████████████████████████████| 15.6 MB 44.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 14.4 MB/s eta 0:00:01
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
     |████████████████████████████████| 462 kB 42.9 MB/s eta 0:00:01
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none

## Binary Classification with Neural Networks
In this notbook, we consider a simple problem in healthcare. Can we predict diabetes based on other healthcare information. For this problem we use **Pima Indians onset of diabetes** dataset from **UCI Machine Learning** repository. This is dataset of structured data.

### Pima Indians Onset of Diabetes Dataset
This dataset has 767 row. The dependent variable, Onset, has values 1 for onset of diabetes and 0 for no sign of the disease.

#### Features
This dataset has 8 input variable. All features are numeric and there is no missing values.

1. Number of times pregnant
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
3. Diastolic blood pressure (mm Hg)
4. Triceps skin fold thickness (mm)
5. 2-Hour serum insulin (mu U/ml)
6. Body mass index (weight in kg/(height in m)^2)
7. Diabetes pedigree function (history in relatives)
8. Age (years)



### Reading the Data

In [104]:
import pandas as pd
df = pd.read_csv('pima-indians-diabetes.data.csv',header=None)
col_list = ['Pregnancy','Glucose','Blood_Pressure','Skin_fold','Insulin','BMI','Relatives_History','Age','Onset']
df.columns = col_list
print(df.shape)
print(df.info())
df.head()

(768, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Pregnancy          768 non-null    int64  
 1   Glucose            768 non-null    int64  
 2   Blood_Pressure     768 non-null    int64  
 3   Skin_fold          768 non-null    int64  
 4   Insulin            768 non-null    int64  
 5   BMI                768 non-null    float64
 6   Relatives_History  768 non-null    float64
 7   Age                768 non-null    int64  
 8   Onset              768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB
None


,Pregnancy,Glucose,Blood_Pressure,Skin_fold,Insulin,BMI,Relatives_History,Age,Onset
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [105]:
# First, we seperate input and output data.
df_y = pd.DataFrame(df['Onset'])
df_x = df[['Pregnancy','Glucose','Blood_Pressure','Skin_fold','Insulin','BMI','Relatives_History','Age']]

### Unbalanced Dataset
There are 500 of 0 labels and 268 of 1 labels, i.e., we have roughly 2 times 0s compared to 1s in this data. This is a problem when we want to choose a threshold to separate 1s from 0s given their probabilities. 

In [196]:
from collections import Counter
print(Counter(df_y['Onset']))

Counter({0: 500, 1: 268})


In [110]:
# Keep some data for out of sample testing

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_x,df_y, test_size=0.2,random_state=42)

### Neural Network Model

Let's start with a feedforward network with 2 hidden layers. TensorFlow has the ability of define each layer as a function. To create the model, we simply compose these functions. The input layer is defined with function Input. Then we have two hidden layers defined by Dense function. The first of these layers has 12 nodes and the next one has 8. 

The Model functions gets the input and output layers as creates a Keras (an internal tensorflow package) model for us. Our model needs a loss function and an optimizer (a version of gradient decent). Here we go with Binary Crossentropy and Adam. The training evaluation metric is accuracy.

The summary function lists the layers of the created model, number of their nodes and number of parameter in each layer. 

This is one of the simplest models that can be defined. It is just a two layer perceptron.

In [111]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense,Input

input1 = Input(shape=(8,))
logits1 = Dense(units=12, activation="relu")(input1)
logits2 = Dense(units=8, activation="relu")(logits1)
#logits2 = Dropout(0.5)(logits1)
logits3 = Dense(units=1, activation="sigmoid")(logits2)

model = Model(inputs=input1, outputs=logits3)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 8)]               0         
_________________________________________________________________
dense_85 (Dense)             (None, 12)                108       
_________________________________________________________________
dense_86 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_87 (Dense)             (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


### Learning Rate
The optimizer function has the default learning rate value. We get this value by running following command.

One can change this value or even define a callback function to change it based on a schedule. 

In [112]:
print(round(model.optimizer.lr.numpy(), 5))

0.001


### Training

Number of epochs and batch size are two of the most important hyper parameters of each neural network model. Epochs determine how many times the model should be retained on the data and batch size determine during each epoch how many of the data points should be considered at each iteration of the epoch.

In [113]:
model.fit(x=X_train, y=y_train,
            validation_split=0.1,
            batch_size=10,
            shuffle=True,
            epochs=10)

Epoch 1/10
56/56 [==============================] - 0s 3ms/step - loss: 12.8158 - accuracy: 0.5199 - val_loss: 4.4228 - val_accuracy: 0.6935
Epoch 2/10
56/56 [==============================] - 0s 2ms/step - loss: 4.6143 - accuracy: 0.6250 - val_loss: 1.5139 - val_accuracy: 0.6935
Epoch 3/10
56/56 [==============================] - 0s 2ms/step - loss: 2.0806 - accuracy: 0.5942 - val_loss: 1.5761 - val_accuracy: 0.6452
Epoch 4/10
56/56 [==============================] - 0s 2ms/step - loss: 1.5242 - accuracy: 0.6141 - val_loss: 1.2501 - val_accuracy: 0.7419
Epoch 5/10
56/56 [==============================] - 0s 2ms/step - loss: 1.2152 - accuracy: 0.6105 - val_loss: 1.0640 - val_accuracy: 0.7097
Epoch 6/10
56/56 [==============================] - 0s 2ms/step - loss: 1.1059 - accuracy: 0.6141 - val_loss: 0.9697 - val_accuracy: 0.6613
Epoch 7/10
56/56 [==============================] - 0s 1ms/step - loss: 0.9773 - accuracy: 0.6159 - val_loss: 1.0739 - val_accuracy: 0.6452
Epoch 8/10
56/56 [=

In [ ]:
### Model e

In [114]:
_, accuracy = model.evaluate(X_test,y_test)
print('Accuracy: %.2f' % (accuracy*100))

5/5 [==============================] - 0s 2ms/step - loss: 0.8778 - accuracy: 0.6234
Accuracy: 62.34


In [115]:
from sklearn.metrics import accuracy_score,confusion_matrix

y_train_pred = (model.predict(X_train) > 0.5).astype(int)
train_acc = accuracy_score(y_train,y_train_pred)
print('Train Accuracy:',round(train_acc * 100,2))

y_pred = (model.predict(X_test) > 0.5).astype(int)
acc = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test,y_pred)

print('Accuracy:',round(acc*100,2))
print('Confusion Matrix:')
print(cm)

Train Accuracy: 62.87
Accuracy: 62.34
Confusion Matrix:
[[61 38]
 [20 35]]


In [116]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[col_list[:-1]] = scaler.fit_transform(df[col_list[:-1]])
print(df.shape)
print(df.info())
df.head()

(768, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Pregnancy          768 non-null    float64
 1   Glucose            768 non-null    float64
 2   Blood_Pressure     768 non-null    float64
 3   Skin_fold          768 non-null    float64
 4   Insulin            768 non-null    float64
 5   BMI                768 non-null    float64
 6   Relatives_History  768 non-null    float64
 7   Age                768 non-null    float64
 8   Onset              768 non-null    int64  
dtypes: float64(8), int64(1)
memory usage: 54.1 KB
None


,Pregnancy,Glucose,Blood_Pressure,Skin_fold,Insulin,BMI,Relatives_History,Age,Onset
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333,1
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667,0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333,1
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000,0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000,1


In [117]:
# First, we seperate input and output data.
df_y = pd.DataFrame(df['Onset'])
df_x = df[['Pregnancy','Glucose','Blood_Pressure','Skin_fold','Insulin','BMI','Relatives_History','Age']]

In [118]:
from collections import Counter
print(Counter(df_y['Onset']))

Counter({0: 500, 1: 268})


In [119]:
# Keep some data for out of sample testing

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_x,df_y, test_size=0.2,random_state=42)

In [192]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense,Dropout,Input

logits1 = Input(shape=(8,))
logits2 = Dense(units=16, activation="relu")(logits1)
logits3 = Dense(units=32, activation="relu")(logits2)
logits4 = Dense(units=32, activation="relu")(logits3)
#logits2 = Dropout(0.5)(logits1)
logits20 = Dense(units=1, activation="sigmoid")(logits4)

model = Model(inputs=logits1, outputs=logits20)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_45 (InputLayer)        [(None, 8)]               0         
_________________________________________________________________
dense_138 (Dense)            (None, 16)                144       
_________________________________________________________________
dense_139 (Dense)            (None, 32)                544       
_________________________________________________________________
dense_140 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_141 (Dense)            (None, 1)                 33        
Total params: 1,777
Trainable params: 1,777
Non-trainable params: 0
_________________________________________________________________


In [193]:
model.fit(x=X_train, y=y_train,
            validation_split=0.1,
            batch_size=10,
            shuffle=True,
            epochs=20)

Epoch 1/20
56/56 [==============================] - 1s 3ms/step - loss: 0.6585 - accuracy: 0.6594 - val_loss: 0.6635 - val_accuracy: 0.5968
Epoch 2/20
56/56 [==============================] - 0s 2ms/step - loss: 0.6311 - accuracy: 0.6612 - val_loss: 0.6519 - val_accuracy: 0.5968
Epoch 3/20
56/56 [==============================] - 0s 2ms/step - loss: 0.6021 - accuracy: 0.6649 - val_loss: 0.6022 - val_accuracy: 0.6290
Epoch 4/20
56/56 [==============================] - 0s 2ms/step - loss: 0.5796 - accuracy: 0.6775 - val_loss: 0.5759 - val_accuracy: 0.7258
Epoch 5/20
56/56 [==============================] - 0s 2ms/step - loss: 0.5567 - accuracy: 0.7120 - val_loss: 0.5683 - val_accuracy: 0.6935
Epoch 6/20
56/56 [==============================] - 0s 2ms/step - loss: 0.5419 - accuracy: 0.7283 - val_loss: 0.5342 - val_accuracy: 0.7258
Epoch 7/20
56/56 [==============================] - 0s 2ms/step - loss: 0.5237 - accuracy: 0.7391 - val_loss: 0.5214 - val_accuracy: 0.7258
Epoch 8/20
56/56 [==

In [194]:
_, accuracy = model.evaluate(X_test,y_test)
print('Accuracy: %.2f' % (accuracy*100))

5/5 [==============================] - 0s 1ms/step - loss: 0.5431 - accuracy: 0.7662
Accuracy: 76.62


In [195]:
from sklearn.metrics import accuracy_score,confusion_matrix

y_train_pred = (model.predict(X_train) > 0.5).astype(int)
train_acc = accuracy_score(y_train,y_train_pred)
print('Train Accuracy:',round(train_acc * 100,2))

y_pred = (model.predict(X_test) > 0.5).astype(int)
acc = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test,y_pred)

print('Accuracy:',round(acc*100,2))
print('Confusion Matrix:')
print(cm)

Train Accuracy: 77.69
Accuracy: 76.62
Confusion Matrix:
[[85 14]
 [22 33]]
